# Data Wrangling for Healthcare Cost Prediction Project

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load the Data

In [2]:
# Load the CSV files
cost_report_data = pd.read_csv('CostReport_2021_Final (1).csv')
data_csv = pd.read_csv('data.csv')

# Initial inspection of the datasets
print(cost_report_data.info())
print(cost_report_data.head())

print(data_csv.info())
print(data_csv.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6051 entries, 0 to 6050
Columns: 117 entries, rpt_rec_num to Stand-Alone CHIP Charges
dtypes: float64(103), int64(4), object(10)
memory usage: 5.4+ MB
None
   rpt_rec_num  Provider CCN                         Hospital Name  \
0       692984        441314         LAUDERDALE COMMUNITY HOSPITAL   
1       695374        341317   LIFEBRITE COMMUNITY HOPITAL OF STOK   
2       695408        511320              JACKSON GENERAL HOSPITAL   
3       695541        520011  LAKEVIEW MEDICAL CENTER OF RICE LAKE   
4       695846        520037             MARSHFIELD MEDICAL CENTER   

           Street Address        City State Code Zip Code      County  \
0           326 ASBURY RD      RIPLEY         TN    38063  LAUDERDALE   
1    1570 NC 8 & 89 HWY N     DANBURY         NC    27016      STOKES   
2     122  PINNELL STREET      RIPLEY         WV    25271     JACKSON   
3  1700 WEST STOUT STREET   RICE LAKE         WI   54868-      BARRON   
4    611

## Clean `data.csv`

In [3]:
# Remove rows where all elements are NaN
data_csv_cleaned = data_csv.dropna(how='all')

# Remove columns where all elements are NaN
data_csv_cleaned = data_csv_cleaned.dropna(axis=1, how='all')

# Rename columns based on the content of the first row if it seems to contain headers
data_csv_cleaned.columns = data_csv_cleaned.iloc[0]
data_csv_cleaned = data_csv_cleaned[1:]  # Remove the first row after setting header

# Display cleaned data.csv
print(data_csv_cleaned.info())
print(data_csv_cleaned.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 1 to 142
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Page    142 non-null    object
 1   #       53 non-null     object
 2   1       29 non-null     object
 3   nan     20 non-null     object
 4   nan     16 non-null     object
 5   nan     10 non-null     object
 6   nan     4 non-null      object
 7   nan     4 non-null      object
 8   nan     2 non-null      object
 9   nan     1 non-null      object
 10  nan     1 non-null      object
dtypes: object(11)
memory usage: 12.3+ KB
None
0     Page     #    1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
1    State  Name  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
2    Total   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
3  ALABAMA   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
4   ALASKA   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
5  ARIZONA   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN


## Further Cleaning of `data.csv`

In [4]:
# Further cleaning of data.csv

# Remove rows where the 'Page' column (now 'State') contains 'Total' or 'NaN'
data_csv_cleaned = data_csv_cleaned[~data_csv_cleaned['Page'].str.contains('Total', na=False)]
data_csv_cleaned = data_csv_cleaned.dropna(subset=['Page'])

# Drop any remaining fully NaN columns again just in case
data_csv_cleaned = data_csv_cleaned.dropna(axis=1, how='all')

# Display the cleaned data again to see the effect
print(data_csv_cleaned.info())
print(data_csv_cleaned.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135 entries, 1 to 142
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Page    135 non-null    object
 1   #       51 non-null     object
 2   1       27 non-null     object
 3   nan     18 non-null     object
 4   nan     14 non-null     object
 5   nan     9 non-null      object
 6   nan     3 non-null      object
 7   nan     4 non-null      object
 8   nan     2 non-null      object
 9   nan     1 non-null      object
 10  nan     1 non-null      object
dtypes: object(11)
memory usage: 12.7+ KB
None
0      Page     #    1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
1     State  Name  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
3   ALABAMA   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
4    ALASKA   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
5   ARIZONA   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
6  ARKANSAS   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN


## Prepare `CostReport_2021_Final (1).csv` for Analysis

In [5]:
# Handle missing values and data types in CostReport_2021_Final (1).csv

# Check for missing values in the cost report data
cost_report_missing = cost_report_data.isnull().sum()
print(cost_report_missing[cost_report_missing > 0].sort_values(ascending=False))

# Handle missing data: Example strategy - remove columns with excessive missing values (over 50%)
threshold = 0.5
cost_report_cleaned = cost_report_data.dropna(thresh=int(threshold * len(cost_report_data)), axis=1)

# Check data types to ensure they are appropriate
print(cost_report_cleaned.info())

Outlier Payments For Discharges                        6051
DRG Amounts Other Than Outlier Payments                6051
Hospital Total Days Title V For Adults & Peds          5926
Hospital Total Discharges Title V For Adults & Peds    5919
Total Discharges Title V                               5919
                                                       ... 
Combined Outpatient + Inpatient Total Charges            75
Rural Versus Urban                                       75
Overhead Non-Salary Costs                                75
Total Costs                                              75
Street Address                                            3
Length: 106, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6051 entries, 0 to 6050
Data columns (total 86 columns):
 #   Column                                                                   Non-Null Count  Dtype  
---  ------                                                                   --------------  -----  
 0 

In [6]:
# Fill missing values for numerical columns using median
numerical_columns = cost_report_cleaned.select_dtypes(include=['float64', 'int64']).columns
cost_report_cleaned[numerical_columns] = cost_report_cleaned[numerical_columns].fillna(cost_report_cleaned[numerical_columns].median())

# Convert date columns to datetime objects if they exist
if 'Fiscal Year Begin Date' in cost_report_cleaned.columns:
    cost_report_cleaned['Fiscal Year Begin Date'] = pd.to_datetime(cost_report_cleaned['Fiscal Year Begin Date'], errors='coerce')
if 'Fiscal Year End Date' in cost_report_cleaned.columns:
    cost_report_cleaned['Fiscal Year End Date'] = pd.to_datetime(cost_report_cleaned['Fiscal Year End Date'], errors='coerce')

# Recheck data structure after final cleaning and preparation
print(cost_report_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6051 entries, 0 to 6050
Data columns (total 86 columns):
 #   Column                                                                   Non-Null Count  Dtype         
---  ------                                                                   --------------  -----         
 0   rpt_rec_num                                                              6051 non-null   int64         
 1   Provider CCN                                                             6051 non-null   int64         
 2   Hospital Name                                                            6051 non-null   object        
 3   Street Address                                                           6048 non-null   object        
 4   City                                                                     6051 non-null   object        
 5   State Code                                                               6051 non-null   object        
 6   Zip Code        

C:\Users\drasw\AppData\Local\Temp\ipykernel_23836\2965723107.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cost_report_cleaned[numerical_columns] = cost_report_cleaned[numerical_columns].fillna(cost_report_cleaned[numerical_columns].median())
C:\Users\drasw\AppData\Local\Temp\ipykernel_23836\2965723107.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cost_report_cleaned['Fiscal Year Begin Date'] = pd.to_datetime(cost_report_cleaned['Fiscal Year Begin Date'], errors='coerce')
C:\Users\drasw\AppData\